# LoRA (Low-Rank Adaptation) - Adaptation par Mise à Jour de Rang Faible

Dans ce notebook, nous allons présenter le concept de **LoRA** et l'illustrer avec un exemple simple en PyTorch.

L'idée de LoRA est de **ne pas mettre à jour l'ensemble des paramètres** d'un modèle pré-entraîné, mais d'ajouter une **mise à jour de rang faible** à certaines matrices de poids. Pour une matrice de poids \(W\), on introduit une mise à jour de la forme :

$$
\Delta W = A \times B,
$$

La matrice finale utilisée lors de l'inférence devient alors :

$$
W_{\text{adapté}} = W + A \times B.
$$

On entraîne uniquement les matrices \(A\) et \(B\) (de petit rang), ce qui permet de réduire le nombre de paramètres entraînables et donc les ressources nécessaires pour l'adaptation.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class LoRALinear(nn.Module):
    def __init__(self, in_features, out_features, r=4):
        """
        in_features  : dimension de l'entrée
        out_features : dimension de la sortie
        r            : rang de la décomposition (nombre de colonnes de A et lignes de B)
        """
        super(LoRALinear, self).__init__()
        
        # Poids pré-entraîné (fixé, non entraînable)
        self.weight = nn.Parameter(torch.randn(out_features, in_features), requires_grad=False)
        self.bias = nn.Parameter(torch.zeros(out_features), requires_grad=False)
        
        # Paramètres de la mise à jour de rang faible (seuls ces paramètres seront entraînés)
        self.A = nn.Parameter(torch.randn(out_features, r))
        self.B = nn.Parameter(torch.randn(r, in_features))
    
    def forward(self, x):
        # Calcul de la mise à jour de rang faible
        delta_W = self.A @ self.B  # Produit matriciel de A et B
        # Calcul de la sortie avec le poids mis à jour
        W_updated = self.weight + delta_W
        return x @ W_updated.t() + self.bias

# Vérification de la définition de la classe
print(LoRALinear)

## Création d'un Jeu de Données Synthétique et Entraînement

Nous allons générer un jeu de données synthétique pour illustrer l'entraînement d'un modèle utilisant LoRA. La relation entre les entrées et les sorties sera linéaire, de la forme :

$$
y = x \times W_{true}^T + b_{true},
$$

Lors de l'entraînement, nous n'ajusterons que les matrices \(A\) et \(B\) afin d'adapter la couche à la tâche.

In [ ]:
# Paramètres du modèle et génération des données
torch.manual_seed(42)
in_features = 10
out_features = 2
r = 2  # Rang faible

# Instanciation du modèle LoRA
model = LoRALinear(in_features, out_features, r)

# Génération de données synthétiques : y = x @ W_true.T + b_true
N = 100  # Nombre d'exemples
x = torch.randn(N, in_features)
W_true = torch.randn(out_features, in_features)
b_true = torch.randn(out_features)
y = x @ W_true.t() + b_true

# Définition de la fonction de perte et de l'optimiseur
# Seuls les paramètres A et B sont entraînés
criterion = nn.MSELoss()
optimizer = optim.SGD([model.A, model.B], lr=0.01)

# Boucle d'entraînement
num_epochs = 200
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(x)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 20 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

## Conclusion

Nous avons ainsi illustré comment le concept de **LoRA** permet d'adapter un modèle pré-entraîné en n'ajustant que quelques paramètres (les matrices \(A\) et \(B\)). Cette approche est particulièrement intéressante pour réduire le coût en ressources et mémoire lors du fine-tuning sur de nouvelles tâches, tout en conservant les paramètres d'origine du modèle.